## Imports

In [2]:
import numpy as np 
import pandas as pd
import urllib
import re
from collections import defaultdict

import os
import shutil

import wikitextprocessor as wtp

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, ArrayType
from pyspark.accumulators import AccumulatorParam

conf = pyspark.SparkConf().setMaster("local[10]").setAll([
                                   ('spark.jars.packages', 'com.databricks:spark-xml_2.12:0.8.0'),
                                   ('spark.executor.memory', '4g'),
                                   ('spark.driver.memory','5g'),
                                   ('spark.driver.maxResultSize', '20G'),
                                   ('spark.executor.heartbeatInterval', '60s'),
                                   ('spark.network.timeout', '61s')
                                  ])
# create the session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# create the context
sc = spark.sparkContext

22/04/24 20:40:29 WARN Utils: Your hostname, iccluster039 resolves to a loopback address: 127.0.1.1; using 10.90.38.15 instead (on interface ens786f0)
22/04/24 20:40:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/salvi/.conda/envs/francesco/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/salvi/.ivy2/cache
The jars for the packages stored in: /home/salvi/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8adc8c95-9f23-4f56-b71c-97e66dd4d6a2;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.8.0 in central
	found commons-io#commons-io;2.6 in central
	found org.glassfish.jaxb#txw2;2.3.2 in central
:: resolution report :: resolve 278ms :: artifacts dl 19ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.8.0 from central in [default]
	commons-io#commons-io;2.6 from central in [default]
	org.glassfish.jaxb#txw2;2.3.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |  

In [2]:
# sc.setLogLevel('DEBUG')

In [3]:
spark

In [4]:
# Inputs

COMMONS_DUMP = '/scratch/WikipediaImagesTaxonomy/commonswiki-20220220-pages-articles-multistream.xml.bz2'
CATEGORIES_DUMP = '/scratch/WikipediaImagesTaxonomy/commonswiki-20220220-categories-multistream.xml.bz2'
# COMMONS_DUMP_REDUCED = '../../commonswiki-20220220-pages-articles-multistream1.xml-p1p1500000.bz2'
# COMMONS_DUMP_REDUCED = '../../commonswiki-20220220-pages-articles-multistream6.xml-p114543930p115400363.bz2'

# list of chunks of the WIT dataset
WIT_DATASET = ['../../wit_v1.train.all-1percent_sample.tsv.gz']

# Outputs (ideally in scratch/, but I don't have permissions)
CATEGORIES_PATH = '/scratch/WikipediaImagesTaxonomy/commonswiki-20220220-category-network.parquet'
FILES_PATH = '/scratch/WikipediaImagesTaxonomy/commonswiki-20220220-files.parquet'

In [5]:
# Adapted from https://github.com/epfl-dlab/WikiPDA/blob/master/PaperAndCode/TopicsExtractionPipeline/GenerateDataframes.py
def normalize_title(title, dumps=True):
    """ Replace _ with space, remove anchor and namespace prefix, capitalize """
    title = urllib.parse.unquote(title)
    if(dumps):
        title = title.split(':', 1)[1]
    title = title.strip()
    if len(title) > 0:
        title = title[0].upper() + title[1:]
    n_title = title.replace("_", " ")
    if '#' in n_title:
        n_title = n_title.split('#')[0]
    return n_title

## Categories

In [6]:
commons_categories_raw = spark.read.format('com.databricks.spark.xml') \
                                .options(rowTag='page').load(COMMONS_DUMP).filter("ns = '14'")
# commons_categories_raw.persist()

In [7]:
commons_categories_raw.printSchema()

root
 |-- id: long (nullable = true)
 |-- ns: long (nullable = true)
 |-- redirect: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _title: string (nullable = true)
 |-- revision: struct (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _deleted: string (nullable = true)
 |    |-- contributor: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _deleted: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- ip: string (nullable = true)
 |    |    |-- username: string (nullable = true)
 |    |-- format: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- minor: string (nullable = true)
 |    |-- model: string (nullable = true)
 |    |-- parentid: long (nullable = true)
 |    |-- sha1: string (nullable = true)
 |    |-- text: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |  

In [10]:
commons_categories_raw.write.mode("overwrite").parquet(CATEGORIES_DUMP)

In [27]:
commons_categories_raw = spark.read.parquet(CATEGORIES_DUMP)

In [28]:
# Build a dictionary of redirects (old_title -> redirect_title)
category_redirects = {normalize_title(r.title): normalize_title(r.redirect._title) 
                      for r in commons_categories_raw.filter('redirect is not null').collect()}
category_redirects

{'Arthur D. Little Building': 'Arthur D. Little Building (MIT Building E60)',
 'Harry K. Thaw': 'Harry Kendall Thaw',
 'Arnold Brunner': 'Arnold W. Brunner',
 'The Netherlands': 'Netherlands',
 'New York State Education Department Building': 'New York State Education Building',
 "Old Saint Mary's Cathedral": "Old St. Mary's Cathedral",
 'SPEEDY': 'Candidates for speedy deletion',
 'Institute of Contemporary Art Boston': 'Institute of Contemporary Art, Boston',
 'Jean Paul Sartre': 'Jean-Paul Sartre',
 'RENAME': 'Media requiring renaming',
 'Greek Orthodox Cathedral of the Annunciation (Boston)': 'Annunciation Greek Orthodox Cathedral of New England',
 'Dornier Delphin': 'Dornier Do L Delphin',
 'Elastane': 'Spandex',
 'Clothing in Elastane': 'Spandex clothing',
 'Images by Beyond My Ken/': 'Images by Beyond My Ken',
 'Earl Hall at Columbia University': 'Earl Hall (Columbia University)',
 'Hamilton Hall': 'Hamilton Hall (Columbia University)',
 'Alumni of the Hunter College': 'Alumni of

In [ ]:
categories_regex = re.compile('(?<!wpTextbox1\.value\+=\')(?<!wpTextbox1\.value=\')\[\[(Category:[^\|]*?)(?:\|.*?)*\]\]')
hiddencat_regex = re.compile('__HIDDENCAT__' + 
                             '|\{\{[hH]iddencat\}\}' + 
                             '|\{\{[uU]ser category.*?\}\}' +
                             '|\{\{[gG]lobal maintenance category\}\}' +
                             '|\[\[(Category:Categories for discussion[^\|]*?)(?:\|.*?)*\]\]' +
                             '|\[\[(Category:Media contributed by[^\|]*?)(?:\|.*?)*\]\]')

In [30]:
class ChildsAccumulator(AccumulatorParam):
    '''
    Accumulator for childs: a dictionary mapping each category to its childs
    '''
    def zero(self, value):
        return defaultdict(list)

    def addInPlace(self, val1, val2):
        for key, value in val2.items():
            val1[key] += value
        return val1

In [31]:
def extract_category(row):
    '''
    Extract the details of a category
    '''
    title = normalize_title(row.title)
    text = row.revision.text._VALUE
        
    parents = re.findall(categories_regex, text) if text else []
    parents = [category_redirects[normalize_title(parent)] if normalize_title(parent) 
               in category_redirects.keys() else normalize_title(parent) for parent in parents]
    global acc
    if parents:
        acc += {parent: [title] for parent in parents}
    return Row(
        id=row.id,
        title=title,
        parents=parents,
        hiddencat=re.search(hiddencat_regex, text) is not None if text else False
    )

In [32]:
# Schema of the processed categories DataFrame
schema_cat = StructType([StructField("id", IntegerType(), True),
                         StructField("title", StringType(), True),
                         StructField("parents", ArrayType(StringType()), True),
                         StructField("hiddencat", BooleanType(), True)])

In [33]:
# We ignore redirect categories, eventually remapping parents to their redirects
acc = sc.accumulator(defaultdict(list), ChildsAccumulator())
categories_clean = spark.createDataFrame(commons_categories_raw.filter('redirect is null')\
                                            .rdd.map(extract_category).filter(lambda r: r is not None), 
                                         schema=schema_cat)

# commons_categories_raw.unpersist()
# categories_clean.persist();

In [34]:
# Workaround for the fact that the value of acc is used before it is filled

TEMP_PATH = '../../dump.xml'

categories_clean.write.format("com.databricks.spark.xml").mode("overwrite")\
                                         .options(rowTag='page', rootTag='pages').save(TEMP_PATH)

# Remove files
shutil.rmtree(TEMP_PATH)

In [35]:
schema_childs = StructType([StructField('title', StringType(), True),
                            StructField('childs', ArrayType(StringType(), True), True)])

In [36]:
childs_df = spark.createDataFrame(acc.value.items(), schema=schema_childs)

In [37]:
categories = categories_clean.alias('c').join(childs_df, categories_clean.title==childs_df.title).select('c.*', 'childs')
# categories_clean.unpersist();
# categories.persist();

In [38]:
categories.write.mode("overwrite").parquet(CATEGORIES_PATH)

22/04/24 20:37:57 WARN TaskSetManager: Stage 17 contains a task of very large size (254960 KiB). The maximum recommended task size is 1000 KiB.


In [5]:
categories = spark.read.parquet(CATEGORIES_PATH)

In [6]:
categories.show()

+---------+--------------------+--------------------+---------+--------------------+
|       id|               title|             parents|hiddencat|              childs|
+---------+--------------------+--------------------+---------+--------------------+
| 23177791|        'Allo 'Allo!|[Television serie...|    false|[David Croft (TV ...|
|115348453|  't Loo (Oldebroek)|[Oldebroek, Popul...|    false|[Landgoed 't Loo ...|
| 27181563|            't Rooth|[Eijsden-Margrate...|    false|[Wayside cross Sm...|
| 12768111|       't Sas, Breda|[Buildings in Bre...|    false|[Voormalige Horec...|
| 97762284|'t Vissertje (Dor...|[Neighbourhoods o...|    false|[Kyck over den Dyck]|
| 70902081|(1′B)+(B1′) locom...|[Locomotives by U...|    false|[(1Bo)+(Bo1) loco...|
| 61949648|               -10°C|                  []|    false|[Thermometers sho...|
|  2535673|         .357 Magnum|[Pistol and rifle...|    false|[Colt Trooper, Co...|
|  3359431|0-6-2+2-6-0 locom...|[Locomotives by W...|    false|[D

In [7]:
hidden_categories = categories.filter('hiddencat is True').select('title').rdd.flatMap(lambda x: x).collect()

In [8]:
len(hidden_categories)

16352

## Files

In [31]:
commons_files_raw = spark.read.format('com.databricks.spark.xml') \
                                .options(rowTag='page').load(COMMONS_DUMP)\
                                .filter("ns = '6'")
# commons_files_raw.persist();

In [32]:
# Build a dictionary of redirects
file_redirects = {normalize_title(r.title): normalize_title(r.redirect._title)
                  for r in commons_files_raw.filter('redirect is not null').collect()}
# file_redirects

For now, we consider only the images that appear in en.wikipedia, discarding all the others. We can also ignore redirects.

In [33]:
wiki_image_names = []

for chunk in WIT_DATASET:
    wiki_image_names += pd.read_csv(chunk, sep="\t").query("language == 'en'")\
                            .image_url.apply(lambda r: normalize_title(r.split('/')[-1], False)).tolist()

In [34]:
# Keep only unique values
wiki_image_names = set(wiki_image_names)

# Remap redirects
wiki_image_names = {file_redirects[name] if name in file_redirects.keys() else name for name in wiki_image_names}

In [35]:
def extract_file(row):
    '''
    Extract the details of a file
    '''
    text = row.revision.text._VALUE

    categories = re.findall(categories_regex, text) if text else []
    
    # No way to do this with a list comprehension (nested conditions work only if there is always an else)
    # Remap categories to their redirect and filter hidden categories
    categories_nohidd = []
    for category in categories:
        category_norm = normalize_title(category)
        if(category_norm not in hidden_categories):
            if(category_norm in category_redirects.keys()):
                if((c:=category_redirects[category_norm]) not in hidden_categories):
                    categories_nohidd.append(c)
            else:
                categories_nohidd.append(category_norm)

    return Row(
        id=row.id,
        title=normalize_title(row.title),
        categories=categories_nohidd
    )

In [36]:
# Schema of the processed files DataFrame
schema_files = StructType([StructField("id", IntegerType(), True),
                           StructField("title", StringType(), True),
                           StructField("categories", ArrayType(StringType()), True)])

In [37]:
# Also for files, we ignore redirects
files = spark.createDataFrame(commons_files_raw.filter('redirect is null')\
                                        .rdd.map(extract_file).filter(lambda r: r is not None), 
                              schema=schema_files)
# commons_files_raw.unpersist();

In [38]:
files.write.mode("overwrite").parquet(FILES_PATH)

In [39]:
files = spark.read.parquet(FILES_PATH)

In [40]:
files.show()

+---------+--------------------+--------------------+
|       id|               title|          categories|
+---------+--------------------+--------------------+
|114792323|PATRICK MAGO (179...|[Players of Mount...|
|114792324|RETURN TO REDFERN...|[Photographs by N...|
|114792325|View from the Sev...|         [Arlingham]|
|114792326|RETURN TO REDFERN...|[Photographs by N...|
|114792328|RETURN TO REDFERN...|[Photographs by N...|
|114792329| Marina Goliasse.jpg|                  []|
|114792330|Downtown Ferndale...|                  []|
|114792331|MATT PLACE (18392...|[Players of Mount...|
|114792332|Benchmark on Citi...|[Richmond, North ...|
|114792334|Portrait photogra...|  [Joséphin Péladan]|
|114792335|SONNY BRISTOW (18...|[Players of Mount...|
|114792336|The River Ayr - g...|[Ayr (civil paris...|
|114792337|MATT PLACE (18370...|[Players of Mount...|
|114792338|Path to Coton - g...|[Coton, Cambridge...|
|114792339|MICHAEL MORRIS (1...|[Players of Mount...|
|114792340|On top of A'Bhuid

## Categories/2

In [41]:
# List of categories that appear in en.wikipedia
categories_in_wikipedia = files.rdd.flatMap(lambda x: x.categories).distinct().map(Row("title")).toDF()
categories_in_wikipedia = categories_in_wikipedia.withColumn('in_en_wiki', lit(True))

In [42]:
categories_in_wikipedia.show(5)

+--------------------+----------+
|               title|in_en_wiki|
+--------------------+----------+
| Santon, Isle of Man|      true|
|Christmas 2020 in...|      true|
|Pacific Sogo Depa...|      true|
|Players (men) by ...|      true|
|Songs of the Beatles|      true|
+--------------------+----------+
only showing top 5 rows



In [43]:
categories_in_wikipedia.count()

190009

In [44]:
categories = categories.alias('c').join(categories_in_wikipedia, 'title', 'left').select('c.*', categories_in_wikipedia.in_en_wiki)
categories = categories.na.fill(False, subset=["in_en_wiki"])

In [45]:
categories.filter('in_en_wiki == True').count()

1746

In [48]:
temp_path = '.'.join(CATEGORIES_PATH.split('.')[:-1]) + '-temp.parquet'
categories.write.mode("overwrite").parquet(temp_path)

shutil.rmtree(CATEGORIES_PATH)
os.rename(temp_path, CATEGORIES_PATH)

## Close

In [ ]:
spark.stop()